## Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# Azure AI Agent with Explicit Settings Example

This notebook demonstrates creating Azure AI Agents with explicit configuration settings rather than relying on environment variable defaults.

## Features Covered:
- Explicit configuration of Azure AI client settings
- Direct specification of project endpoint and model deployment
- Custom agent naming during client creation
- Alternative to environment variable-based configuration

## Prerequisites

Before running this notebook, ensure you have:
- Azure CLI installed and authenticated (`az login --use-device-code`)
- Access to an Azure AI Foundry project with deployed models
- Environment variables set up (see Initial Setup section below)

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [1]:
import asyncio
import os
from random import randint
from typing import Annotated
import os
from pathlib import Path

from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field
from dotenv import load_dotenv  # For loading environment variables from .env file

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file

False

In [3]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

In [4]:
async def main() -> None:
    print("=== Azure AI Chat Client with Explicit Settings ===")

    # Since no Agent ID is provided, the agent will be automatically created
    # and deleted after getting a response
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(
                project_endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"],
                model_deployment_name=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
                async_credential=credential,
                agent_name="WeatherAgent",
            ),
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        result = await agent.run("What's the weather like in New York?")
        print(f"Result: {result}\n")

# Run the main function
await main()

=== Azure AI Chat Client with Explicit Settings ===


[2026-02-11 16:37:28 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Result: The weather in New York is sunny with a high of 20°C.

